<a href="https://colab.research.google.com/github/AlmaBasic/Predicting/blob/main/Neuralne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Korak 1: Importovanje biblioteka

In [ ]:
!pip install torch torchvision

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

Korak 2: Importovanje dataseta

In [ ]:
diseases = pd.read_csv('disease.csv')

drug = pd.read_csv('drug.csv')

association = pd.read_csv('drug-disease association.csv')

Korak 3: Analiza podataka

Korak 3.1: Analiza tabele bolesti

In [ ]:
diseases
#diseases.shape #(2834, 3)

#diseases.describe(include=['O'])
#diseases.describe(include='all')
#diseases.dropna(axis="rows", subset="name", inplace=True)
diseases
# diseases.info() # svi imaju id, 2827 nema ime, 2418 nema alias
# diseases["name"].value_counts()
# diseases["alias"].value_counts()
# diseases["name"] == ""
# diseases.loc[diseases[1] == ""]
# diseases.loc[:,diseases.iloc[2].isnull()]

,disease_id,name,alias
0,C531616,NaN,NaN
1,C531617,Amyotrophic lateral sclerosis 1,"ALS1#AMYOTROPHIC LATERAL SCLEROSIS 1, AUTOSOMA..."
2,C531755,Kidney disorder involving deposition of calciu...,NaN
3,C531767,NaN,NaN
4,C531795,Idiopathic intracranial hypertension with papi...,NaN
...,...,...,...
2829,D065666,Mesenteric Ischemia,Acute Mesenteric Arterial Embolus#Acute Mesent...
2830,D065766,Atypical Hemolytic Uremic Syndrome,"AHUS1#AHUS2#AHUS3#AHUS4#AHUS5#AHUS6#AHUS, SUSC..."
2831,D065817,Congenital Microtia,Anotia#Anotias#Congenital Microtias#Microtia#M...
2832,D065886,Neurodevelopmental Disorders,Child Mental Disorder#Child Mental Disorders#D...


Korak 3.2: Analiza tabele lijekova

In [ ]:
drug
#drug.shape
drug.info()
#drug.dropna(axis="rows", subset="name", inplace=True)
#drug


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1323 entries, 0 to 1322
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   drug_id      1323 non-null   object 
 1   name         1321 non-null   object 
 2   drugbank_id  1319 non-null   object 
 3   pubchem_cid  1305 non-null   float64
 4   alias        1224 non-null   object 
dtypes: float64(1), object(4)
memory usage: 51.8+ KB


,drug_id,name,drugbank_id,pubchem_cid,alias
0,C074619,bivalirudin,DB00006,16129704.0,Angiomax#BG8967#BG 8967#BG-8967#Hirulog#Hirulo...
1,D017273,Goserelin,DB00014,5311128.0,"Acetate, Goserelin#Goserelin Acetate#ICI118630..."
2,D003894,Deamino Arginine Vasopressin,DB00035,27991.0,1-Deamino-8-D-arginine Vasopressin#1-Desamino-...
3,C062876,cetrorelix,DB00050,25074887.0,cetrorelix acetate#cetrorelix pamoate#Cetrotid...
4,D005259,Felypressin,DB00093,5956.0,"Lysine Vasopressin, Phenylalanine#Octapressin#..."
...,...,...,...,...,...
1318,D003534,Cyproterone,DB04839,5284537.0,NaN
1319,D003647,Debrisoquin,DB04840,2966.0,Debrisoquine#Tendor
1320,D005444,Flunarizine,DB04841,941361.0,"Dihydrochloride, Flunarizine#Flunarizin#Flunar..."
1321,D005484,Fluspirilene,DB04842,3396.0,"Beta, Fluspirilen#Fluspi#Fluspirilen Beta#Flus..."


Korak 3.3: Analiza tabele veza izmedju bolesti i lijekova

In [ ]:
#association
#association.shape
association.info()
association
#sns.countplot(data=association, x='disease_name')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49217 entries, 0 to 49216
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   drug_id       49217 non-null  object
 1   drug_name     49142 non-null  object
 2   disease_id    49217 non-null  object
 3   disease_name  49200 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


,drug_id,drug_name,disease_id,disease_name
0,C074619,bivalirudin,MESH:D003328,Coronary Thrombosis
1,C074619,bivalirudin,MESH:D006470,Hemorrhage
2,C074619,bivalirudin,MESH:D007249,Inflammation
3,C074619,bivalirudin,MESH:D007511,Ischemia
4,C074619,bivalirudin,MESH:D013921,Thrombocytopenia
...,...,...,...,...
49212,D013747,Tetrabenazine,MESH:D017109,"Akathisia, Drug-Induced"
49213,D013747,Tetrabenazine,MESH:D018476,Hypokinesia
49214,D013747,Tetrabenazine,MESH:D020734,Parkinsonian Disorders
49215,D013747,Tetrabenazine,MESH:D020820,Dyskinesias


Korak 4: Kreiranje dataseta za treniranje i testiranje

In [ ]:
#Podjeliti podatke

train_dataset = TensorDataset(diseases, association)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

ValueError: Found input variables with inconsistent numbers of samples: [2834, 1323]

Korak 5: Transformacija podataka u tenzore

In [ ]:
#set_za_obucavanje_tenzor=torch.FloatTensor(set_za_obucavanje1)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

Korak 6: Kreiranje arhitekture dvostrukog varijacionog autoenkodera - DVAE

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim) # Mean μ
        self.fc22 = nn.Linear(hidden_dim, latent_dim) # Log variance σ^2

        # Decoder
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, input_dim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [ ]:
# Rekonstukcija + KL divergencija gubitka sumirana kroz sve elemente i batch-eve
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, input_dim), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD


In [ ]:
input_dim = 3
hidden_dim = 400
latent_dim = 20

model = VAE(input_dim, hidden_dim, latent_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


VAE(
  (fc1): Linear(in_features=3, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=3, bias=True)
)

In [ ]:
train_dataset = diseases.transform
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

TypeError: object of type 'method' has no len()

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}")

    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')


In [ ]:
epochs = 10
log_interval = 10
for epoch in range(1, epochs + 1):
    train(epoch)

NameError: name 'train_loader' is not defined